In [13]:
import os
import sqlite3
import torch
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import networkx as nx
import openai

# Keyword Extractor
class KeywordExtractor:
    def __init__(self, method="roberta"):
        self.method = method.lower()
        if self.method == "roberta":
            self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
            self.model = RobertaModel.from_pretrained("roberta-base")
        elif self.method == "ada":
            self.model = "ada"  # Ada doesn't use tokenizer/model directly
        else:
            raise ValueError("Supported methods are 'roberta' and 'ada'.")

    def extract_keywords(self, query):
        return query.split()  # Simple split (enhance as needed)

# Embedding Handler
class EmbeddingHandler:
    def __init__(self, method="roberta"):
        self.method = method.lower()
        if self.method == "roberta":
            self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
            self.model = RobertaModel.from_pretrained("roberta-base")
        elif self.method == "ada":
            self.model = "ada"
        else:
            raise ValueError("Supported methods are 'roberta' and 'ada'.")

    def get_embedding(self, text):
        if self.method == "roberta":
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True)
            with torch.no_grad():
                outputs = self.model(**inputs)
            return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        elif self.method == "ada":
            try:
                response = openai.Embedding.create(
                    input=text,
                    model="text-embedding-ada-002"
                )
                return np.array(response['data'][0]['embedding'])
            except openai.error.OpenAIError as e:
                print(f"OpenAI API error: {e}")
                return None

    def calculate_similarity(self, emb1, emb2):
        return cosine_similarity([emb1], [emb2])[0][0]

# Schema Extractor
class SchemaExtractor:
    def __init__(self, db_file):
        self.db_file = db_file

    def extract_schema(self):
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        schema = {}
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        for table in tables:
            table_name = table[0]
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            schema[table_name] = [col[1] for col in columns]
        conn.close()
        return schema

# Connections
class Connections:
    def __init__(self, db_file):
        self.db_file = db_file

    def extract_keys(self, table_name):
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        primary_keys = []
        foreign_keys = []

        # Extract primary keys
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        for col in columns:
            if col[-1] == 1:  # Primary key indicator
                primary_keys.append(col[1])

        # Extract foreign keys
        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        keys = cursor.fetchall()
        for key in keys:
            foreign_keys.append({
                "from_column": key[3],
                "to_table": key[2],
                "to_column": key[4]
            })

        conn.close()
        return primary_keys, foreign_keys

# RAG Pipeline
class RAGPipeline:
    def __init__(self, db_file, user_query, similarity_threshold=0.4, embedding_method="roberta"):
        self.db_file = db_file
        self.user_query = user_query
        self.similarity_threshold = similarity_threshold
        self.embedding_handler = EmbeddingHandler(method=embedding_method)
        self.keyword_extractor = KeywordExtractor(method=embedding_method)
        self.schema_extractor = SchemaExtractor(db_file)
        self.schema = self.schema_extractor.extract_schema()
        self.connections = Connections(db_file)

    def identify_relevant_tables(self, keywords):
        results = []
        for keyword in keywords:
            keyword_embedding = self.embedding_handler.get_embedding(keyword)
            tables = []
            similarities = []
            for table, columns in self.schema.items():
                table_embedding = self.embedding_handler.get_embedding(table)
                similarity = self.embedding_handler.calculate_similarity(
                    table_embedding, keyword_embedding
                )
                if similarity >= self.similarity_threshold:
                    tables.append(table)
                    similarities.append(similarity)
            results.append({
                "Keyword": keyword,
                "Identified Tables": tables,
                "Similarity Scores": similarities
            })
        return pd.DataFrame(results)

    def identify_relevant_columns(self, keywords):
        results = []
        for keyword in keywords:
            keyword_embedding = self.embedding_handler.get_embedding(keyword)
            columns = []
            similarities = []
            for table, cols in self.schema.items():
                for column in cols:
                    column_name = f"{table}.{column}"
                    column_embedding = self.embedding_handler.get_embedding(column_name)
                    similarity = self.embedding_handler.calculate_similarity(
                        column_embedding, keyword_embedding
                    )
                    if similarity >= self.similarity_threshold:
                        columns.append(column_name)
                        similarities.append(similarity)
            results.append({
                "Keyword": keyword,
                "Identified Columns": columns,
                "Similarity Scores": similarities
            })
        return pd.DataFrame(results)

    def build_subgraph(self, table_similarity_df, column_similarity_df):
        graph = nx.Graph()

        # Add nodes for tables
        for index, row in table_similarity_df.iterrows():
            for table in row["Identified Tables"]:
                graph.add_node(table, type="table")

        # Add nodes for columns and edges to their tables
        for index, row in column_similarity_df.iterrows():
            for column in row["Identified Columns"]:
                table_name = column.split('.')[0]
                graph.add_node(column, type="column")
                graph.add_edge(table_name, column)

        # Add edges between tables based on foreign keys
        for table in self.schema.keys():
            _, foreign_keys = self.connections.extract_keys(table)
            for fk in foreign_keys:
                graph.add_edge(table, fk["to_table"],
                               from_column=fk["from_column"],
                               to_column=fk["to_column"])

        print("Subgraph Nodes:")
        print(graph.nodes(data=True))

        print("Subgraph Edges:")
        print(graph.edges(data=True))

        return graph

    def run(self):
        print("Extracting keywords...")
        keywords = self.keyword_extractor.extract_keywords(self.user_query)
        print(f"Keywords: {keywords}")

        print("Identifying relevant tables...")
        table_similarity_df = self.identify_relevant_tables(keywords)
        print(table_similarity_df)

        print("Identifying relevant columns...")
        column_similarity_df = self.identify_relevant_columns(keywords)
        print(column_similarity_df)

        print("Building subgraph...")
        subgraph = self.build_subgraph(table_similarity_df, column_similarity_df)

        return table_similarity_df, column_similarity_df, subgraph
variable = 1
# Main Runner
if __name__ == "__main__":
    # Set OpenAI API Key
    openai.api_key = os.getenv("OPENAI_API_KEY")
    if not openai.api_key:
        raise ValueError("OpenAI API Key is not set. Please set it as an environment variable.")

    DB_FILE = "company_data.db"
    USER_QUERY = "How many people have booked at least 50 hours before 01/05/2024?"
    EMBEDDING_METHOD = "ada"  # Change to "roberta" or "ada"

    pipeline = RAGPipeline(DB_FILE, USER_QUERY, embedding_method=EMBEDDING_METHOD)
    table_similarity, column_similarity, subgraph = pipeline.run()


ValueError: OpenAI API Key is not set. Please set it as an environment variable.